# MBIT School

## Master Big Data Cloud y Analytics

---

## Módulo 2: Ecosistema Spark
## Submódulo DC02: Ecosistema Spark

---

### Taller Spark + Kafka: Gestor de cuentas (multithread/consumidor)

#### Jacinto Arias (arias.jacinto@gmail.com)

---


En esta libreta implementaremos un proceso consumidor de kafka que procesará los mensajes del stream y mantendrá actualizada el estado de una "supuesta base de datos" con los totales de la cuenta conforme hayamos procesado los mensajes.. Esta libreta implementa un consumidor en paralelo que permite aprovechar el paralelismo de un cluster de kafka

#### Imports

In [ ]:
# python
from collections import defaultdict
import json
import threading

In [ ]:
# jupyter
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
# kafka

from confluent_kafka import Producer, Consumer

#### Constantes

In [ ]:
KAFKA_BOOTSTRAP_SERVERS = "localhost:9092"

In [ ]:
KAFKA_CONSUMER_GROUP = "fake_bank.account_manager"
KAFKA_TOPIC_TRANSACTIONS = "fake_bank.transactions"

#### Hilo Consumidor 

Este sistema nos permite lanzar aplicaciones multihilo en python

In [ ]:
class ConsumerThread(threading.Thread):
    
    def __init__(self, out):
        super(ConsumerThread, self).__init__()
        
        self.out = out
        
        self.kafka_consumer = Consumer({
            'bootstrap.servers': KAFKA_BOOTSTRAP_SERVERS,
            'group.id': KAFKA_CONSUMER_GROUP
        })
            
        self.account_db = defaultdict(int)

        self.kafka_consumer.subscribe([KAFKA_TOPIC_TRANSACTIONS])

        self.active = True
        
        
    def run(self):
        while self.active:

            transaction = self.kafka_consumer.poll(1.0)

            if transaction is None:
                continue

            payload = transaction.value().decode('utf-8')
            data = json.loads(payload)

            self.account_db[data["account"]] += data["amount"]        
            self.out.clear_output()
            self.out.append_display_data(self.account_db)

        self.kafka_consumer.close()
        
        
    def stop(self):
        self.active = False
        

#### Creacion de hilos y salidas

Utilizaremos múltiples salidas en nuestros hilos para verlos por separado

In [ ]:
out_1 = widgets.Output()
out_2 = widgets.Output()
out_3 = widgets.Output()

consumer_1 = ConsumerThread(out_1)
consumer_1.start()

consumer_2 = ConsumerThread(out_2)
consumer_2.start()


consumer_3 = ConsumerThread(out_3)
consumer_3.start()

In [ ]:
display(out_1)

In [ ]:
display(out_2)

In [ ]:
display(out_3)

#### Importante apagar los hilos!!!

In [ ]:
consumer_1.stop()
consumer_2.stop()
consumer_3.stop()